In [1]:
import numpy as np
import pandas as pd
import os
import string
from unidecode import unidecode
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from transformers import AdamW, BertModel, AutoConfig, AutoTokenizer
from torch.optim import Adam
import torch.nn as nn
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
import time;
import datetime
from sklearn.model_selection import train_test_split
from torchcrf import CRF

/home/oeren/Downloads/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [68]:
conda install pytorch-crf

Channels:
 - defaults
 - conda-forge
 - pytorch
Platform: linux-64
/ 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# !pip install pytorch-crf
# !pip install unidecode
# !pip install --upgrade transformers
# !pip install --upgrade torch

In [104]:
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [105]:
# nlp_train=pd.read_csv("/kaggle/input/nlp-project-train/train.csv", index_col=[0])
# nlp_test=pd.read_csv("/kaggle/input/nlp-project-train/test.csv",index_col=[0],encoding="windows-1252") 

nlp_train=pd.read_csv("train.csv", index_col=[0])
nlp_test=pd.read_csv("test.csv",index_col=[0],encoding="windows-1252") 
# nlp_train = nlp_train[:1000]

In [106]:
def convert_to_ascii(sentence):
    text = unidecode(sentence)
    return text

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
        text = text.replace('  ', ' ')
    return text.strip()

In [107]:
def split_sentences(sentences, max_length=100):
    punctuations = {'.', ',', ';', ':'}
    results = []

    for sentence in sentences:
        while len(sentence) > max_length:
            # find last punctuation before max_length
            split_pos = -1
            for p in punctuations:
                pos = sentence.rfind(p, 0, max_length + 1)
                if pos > split_pos:
                    split_pos = pos
            
            # If no punctuation found, split at the last space before max_length
            if split_pos == -1:
                split_pos = sentence.rfind(' ', 0, max_length + 1)
            
            # If no space found, just split at max_length
            if split_pos == -1:
                split_pos = max_length
            
            # Append the split segment to results
            results.append(sentence[:split_pos + 1].strip())
            # Move the rest of the sentence forward
            sentence = sentence[split_pos + 1:].strip()
        
        # Append the remainder of the sentence if it's not empty
        if sentence:
            results.append(sentence)
    
    return np.array(results)

In [108]:
def encode_word(sentence):
    words = sentence.split()
    encoded_sentence = []
    for word in words:
        encoded_word = []
        for char in word:
            # print(char)
            if char in "ıöüğşç":
                encoded_word.append(2)
            else:
                encoded_word.append(1)
            # print(encoded_word)
        encoded_sentence.append(encoded_word)
    return encoded_sentence

def padding(text, filling_char, max_length):
    if type(text) is str:
        text = text + filling_char * (max_length - len(text))
    elif type(text) is list:
        text = text + [filling_char] * (max_length - len(text))
    return text

In [109]:
import re

def remove_puncutations_and_numbers(text):
    # Iterate over the string and remove char if it is not a character
    characters = "abcçdefgğhıijklmnoöprsştuüvyzABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZxXwWqQ "
    for char in text:
        if char not in characters:
            text = text.replace(char, "")
    # Remove multiple spaces
    text = re.sub(' +', ' ', text)
    # Lowercase the text
    text = text.lower()
    return text

In [110]:
# Raw sentences
raw_sentences = nlp_train["Sentence"].values
# Split sentences
sentences = split_sentences(raw_sentences, max_length=99)
# Remove punctuations
# sentences = [remove_punctuations(s).lower() for s in sentences]
sentences = [remove_puncutations_and_numbers(i) for i in sentences]
nlp_train["Label"] = nlp_train["Sentence"]
# Apply convert to ascii to y_train
asci_sentences = [convert_to_ascii(s.lower()) for s in sentences]

In [111]:
def map_diacritics(text):
    for char_index in range(len(text)):
        # print(text[char_index])
        if text[char_index] in "ıöüğşç":
            text[char_index] = 2
        elif text[char_index] in "iougsc":
            text[char_index] = 1
        elif text[char_index] == " ":
            text[char_index] = 0
        else:
            text[char_index] = 3
        
    return text

In [112]:
def lower_label(text):
    for char_index in range(len(text)):
        if text[char_index] == "I":
            text[char_index] = "ı"
        else:
            text[char_index] = text[char_index].lower()
    return text    

In [113]:
# Convert sentences to list of characters
char_sentences_original = [lower_label(list(s)) for s in sentences]


# Apply map_diacritics to the numeric sentences
labels = [map_diacritics(sentence.copy()) for sentence in char_sentences_original]

char_sentences = [list(s) for s in asci_sentences]


In [114]:
characters_dictionary = {
    "a":1,
    "b":2,
    "c":3,
    "ç":4,
    "d":5,
    "e":6,
    "f":7,
    "g":8,
    "ğ":9,
    "h":10,
    "ı":11,
    "i":12,
    "j":13,
    "k":14,
    "l":15,
    "m":16,
    "n":17,
    "o":18,
    "ö":19,
    "p":20,
    "r":21,
    "s":22,
    "ş":23,
    "t":24,
    "u":25,
    "ü":26,
    "v":27,
    "y":28,
    "z":29,
    "q": 30,
    "x":31,
    "w":32,
    " ":33    
}

In [115]:
# Copy char_sentences to numeric_sentences
numeric_sentences = [sentence.copy() for sentence in char_sentences]
# Map each character to an integer in the sentences array to its value in the characters dictionary
for sentence_index in range(len(char_sentences)):
    for char_index in range(len(char_sentences[sentence_index])):
        # print(characters_dictionary[char_sentences[sentence_index][char_index]])
        numeric_sentences[sentence_index][char_index] = characters_dictionary[char_sentences[sentence_index][char_index]]

In [116]:
# Apply padding to the numeric sentences
max_length = 100
for sentence_index in range(len(numeric_sentences)):
    numeric_sentences[sentence_index] = padding(numeric_sentences[sentence_index], 0, max_length)

In [117]:
# Apply padding to the labels
max_length = 100
for sentence_index in range(len(labels)):
    labels[sentence_index] = padding(labels[sentence_index], 0, max_length)

In [200]:
import torch
import torch.nn as nn
from torchcrf import CRF

class DiacritizationBiLSTMCRF(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels):
        super(DiacritizationBiLSTMCRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim // 2, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(hidden_dim, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, sentences, labels=None):
        # Embedding layer
        x = self.embedding(sentences)
        
        # BiLSTM layer
        x, _ = self.lstm(x)
        x = self.dropout(x)
        
        # Fully connected layer
        emissions = self.fc(x)
        # CRF layer
        if labels is not None:
            # If labels are provided, calculate the loss
            loss = -self.crf(emissions, labels)
            return loss
        else:
            # Otherwise, return the best path
            prediction = self.crf.decode(emissions)
            # get probabilites of the best path
            return prediction

# Example of initializing the model
vocab_size = len(characters_dictionary) + 1  # Number of unique characters in your character dictionary + 1 for padding
embed_dim = 128
hidden_dim = 256
num_labels = 4  # Diacritic or not
model = DiacritizationBiLSTMCRF(vocab_size, embed_dim, hidden_dim, num_labels)
model.to(device)


DiacritizationBiLSTMCRF(
  (embedding): Embedding(34, 128)
  (lstm): LSTM(128, 128, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=256, out_features=4, bias=True)
  (crf): CRF(num_tags=4)
)

In [201]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Assuming 'numeric_sentences' and 'labels' are your preprocessed datasets available as lists of lists
# Convert them into tensors
numeric_sentences_tensor = torch.tensor(numeric_sentences, dtype=torch.long)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

# Splitting three variables: numeric_sentences_tensor, labels_tensor, and another_tensor
X_train, X_val, y_train, y_val, sentences_train, sentences_val = train_test_split(
    numeric_sentences_tensor, labels_tensor, asci_sentences, test_size=0.2, random_state=42)

# Create TensorDataset and DataLoader for training and validation
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# Define the loss function and the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training the model
def train_model(model, data_loader, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for sentences, label_seqs in data_loader:
            sentences, label_seqs = sentences.to(device), label_seqs.to(device)
            
            # Forward pass: Compute predicted y by passing x to the model
            loss = model(sentences, labels=label_seqs)
            
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")

# Start training
#train_model(model, train_loader, optimizer, num_epochs=10)


In [240]:

def preprocess_sentence(sentence, characters_dictionary, max_length=200):
    # Remove punctuation and numbers
    sentence = ''.join([i for i in sentence if not i.isdigit() and i not in string.punctuation])
    sentence = sentence.lower().strip()
    
    # Convert to numeric form using the character dictionary
    numeric_sentence = [characters_dictionary.get(char, 0) for char in sentence]  # default to 0 if char not found
    
    # Padding
    if len(numeric_sentence) > max_length:
        numeric_sentence = numeric_sentence[:max_length]
    else:
        numeric_sentence += [0] * (max_length - len(numeric_sentence))
    
    return torch.tensor([numeric_sentence], dtype=torch.long).to(device)

def predict(model, sentence, characters_dictionary):
    model.eval()
    with torch.no_grad():
        numeric_sentence = preprocess_sentence(sentence, characters_dictionary)
        prediction = model(numeric_sentence)
        return prediction[0]  # since we have a batch size of 1

# Example sentence
test_sentence = "bu standart sualtinda kendini rahat hisseden herkesin sportif dalici olabilecegini"
model = DiacritizationBiLSTMCRF(vocab_size, embed_dim, hidden_dim, num_labels)
model.to(device)
# Load the saved weights
model.load_state_dict(torch.load('/home/oeren/Documents/YZV-NLP/weights_0001/model_epoch_29.pth', map_location=torch.device('cpu')))
# Predict using the model
predicted_labels = predict(model, test_sentence, characters_dictionary)
print(predicted_labels)

# Define labels to diacritics mapping
def labels_to_diacritics(text_sentence, labels):
    diacritics_map_dict = {"i": "ı", "o": "ö", "u": "ü", "g": "ğ", "s": "ş", "c": "ç"}
    output_sentence = ""
    for i in range(len(text_sentence)):
        if labels[i] == 2:
            if text_sentence[i] in diacritics_map_dict:
                output_sentence += diacritics_map_dict[text_sentence[i]]
            else:
                output_sentence += text_sentence[i]
        else:
            output_sentence += text_sentence[i]
    return output_sentence

# Show the results
output_sentence = labels_to_diacritics(test_sentence, predicted_labels)
print("Original:", test_sentence)
print("Processed:", output_sentence)

[3, 1, 0, 1, 3, 3, 3, 3, 3, 3, 3, 0, 1, 1, 3, 3, 3, 2, 3, 3, 3, 0, 3, 3, 3, 3, 1, 3, 1, 0, 3, 3, 3, 3, 3, 0, 3, 1, 1, 1, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 1, 1, 3, 0, 1, 3, 1, 3, 3, 1, 3, 0, 3, 3, 3, 2, 1, 2, 0, 1, 3, 3, 3, 1, 3, 3, 1, 3, 2, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Original: bu standart sualtinda kendini rahat hisseden herkesin sportif dalici olabilecegini
Processed: bu standart sualtında kendini rahat hisseden herkesin sportif dalıcı olabileceğini


In [241]:
len("hayatı boyunca otuz kadar eser vermiş ve yüzlerce makale yazmış olan hüseyin nihal atsız hoca`mızın bütün eserlerini uluslararası ağa döküp , dünya çapında isteyen herkesin ulaşabileceği bi.")

190

In [242]:
import zemberek
from zemberek.morphology import TurkishMorphology
from zemberek.morphology.analysis.rule_based_analyzer import RuleBasedAnalyzer
from zemberek.normalization import TurkishSpellChecker
from zemberek.morphology.morphotactics import TurkishMorphotactics, InformalTurkishMorphotactics
#import RootLexicon
from zemberek.morphology.lexicon import RootLexicon

In [243]:
def acc_overall(test_result, testgold):
  
  correct = 0
  total = 0
  # count number of correctly diacritized words
  for i in range(len(testgold)):
    for m in range(len(testgold[i].split())):
      if test_result[i].split()[m] == testgold[i].split()[m]:
        correct += 1
      total +=1

  return correct / total

In [244]:
diacritic_versions = {"i":"ı", "ı":"i", "o":"ö", "ö":"o", "u":"ü", "ü":"u", "g":"ğ", "ğ":"g", "s":"ş", "ş":"s", "c":"ç", "ç":"c"}

In [246]:
def check_morphology(sentences):
    morphology = TurkishMorphology.create_with_defaults()
    morphology.ignoreDiacriticsInAnalysis = False
    spellChecker = TurkishSpellChecker(morphology)

    corrected_sentences = sentences.copy()

    # Iterate through each sentence
    for i in range(len(corrected_sentences)):
        # Split the sentence into words and iterate through each word
        words = corrected_sentences[i].split()
        for m in range(len(words)):
            analysis = morphology.analyze(words[m])

            # Check if the word has no analysis results
            if len(analysis.analysis_results) == 0:
                print("No analysis results for:", words[m])
                
                # Check if there are suggestions from the spell checker
                suggestions = spellChecker.suggest_for_word(words[m])
                if len(suggestions) != 0:
                    for suggested_word in suggestions:
                        print("Suggested word:", suggested_word)                  
                        # if the suggested word has the same length as the original word, and just "ıioöuügğsşcç" characters are different, replace the word
                        if len(suggested_word) == len(words[m]):
                            for char1, char2 in zip(suggested_word, words[m]):
                                if char1 in diacritic_versions and char2 == diacritic_versions[char1]:
                                    continue
                                if char1 in diacritic_versions and char2 == char1:
                                    continue
                                if char1 not in diacritic_versions and char1 == char2:
                                    continue
                                elif char1 in diacritic_versions and char2 != diacritic_versions[char1] and char2 != char1:
                                    break
                                elif char1 not in diacritic_versions and char1 != char2:
                                    break
                            else:
                                print("Suggested word is approved:", suggested_word)
                                words[m] = suggested_word
                                break

        # Join the modified words back into a sentence
        corrected_sentences[i] = ' '.join(words)

    return corrected_sentences

def test_model(model, val_loader, characters_dictionary):

    model.eval()  # Set the model to evaluation mode
    predicted_results = []
    true_results = []

    for (sentences, label_seqs), real_sentence in zip(val_loader, sentences_val):
        sentence = sentences.squeeze(0)  # Remove the batch dimension
        sentence = sentence.tolist()  # Convert tensor to list of sentences
        # Preprocess each sentence and obtain predicted labels
        model.eval()
        with torch.no_grad():
            prediction = model(sentences)
        predicted_sentence = labels_to_diacritics(real_sentence, prediction[0])
        
        # Append predicted and true results
        predicted_results.append(predicted_sentence)
        true_results.append(labels_to_diacritics(real_sentence, label_seqs.squeeze(0).tolist()))

    last_version = check_morphology(predicted_results)
    # Calculate accuracy
    accuracy = acc_overall(last_version, true_results)
    return accuracy

# Test the model on the validation dataset
accuracy = test_model(model, val_loader, characters_dictionary)
print("Validation Accuracy:", accuracy)


2024-05-06 17:54:25,343 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 5.7880096435546875

No analysis results for: q
Suggested word: o
Suggested word: a
Suggested word: e
Suggested word: b
Suggested word: c
Suggested word: d
Suggested word: s
Suggested word: ı
Suggested word: g
Suggested word: u
Suggested word: f
Suggested word: k
Suggested word: m
Suggested word: p
Suggested word: v
Suggested word: h
Suggested word: t
Suggested word: y
Suggested word: r
Suggested word: n
Suggested word: z
Suggested word: ç
Suggested word: ö
Suggested word: ş
Suggested word: ıq
Suggested word: j
Suggested word: q7
Suggested word: qi
No analysis results for: q
Suggested word: o
Suggested word: a
Suggested word: e
Suggested word: b
Suggested word: c
Suggested word: d
Suggested word: s
Suggested word: ı
Suggested word: g
Suggested word: u
Suggested word: f
Suggested word: k
Suggested word: m
Suggested word: p
Suggested word: v
Suggested word: h
Suggested wo

In [34]:
0.9544237320915839